Install and import all the necessary packages:

In [3]:
%pip install astroplan
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import astroplan
from datetime import datetime
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_body, get_sun, Angle
from astropy.utils import iers
from ipywidgets import interact, FloatSlider
%matplotlib inline

Upload text file where all the stars are (downloaded from https://simbad.cds.unistra.fr/simbad/sim-fsam where Vmag is less than 6.5)

In [5]:
file_path = 'stars.tsv'

df = pd.read_table(file_path, sep='\t', skiprows=5, header=1, skipinitialspace=True, engine='python', index_col=0, skipfooter=2)
df.drop(df.index[0], inplace = True)
df.columns
df.rename(columns={'identifier             ':'identifier' ,'coord1 (ICRS,J2000/2000)      ': 'coordinates', 'Mag V  ': 'Mag_V', 'all types                            ':'types'}, inplace=True)
df[['RA', 'Dec']] = df['coordinates'].str.extract(r'(\d+\.\d+) ([+-]?\d+\.\d+)')
df.drop(['typ', 'coordinates'], axis=1, inplace=True)
df = df[df[['RA','Dec']].notna().all(axis=1)]
df = df.astype({'RA': float, 'Dec': float, 'Mag_V': float})
pattern = r'(?:^\*$|,\*|,\*,|^\*,*)'
df = df[df['types'].str.contains(pattern, regex=True)]
df

,identifier,types,Mag_V,RA,Dec
#,,,,,
1,HD 200253,"NIR,*,IR ...",5.997,315.303603,36.026012
2,* 61 Cyg B,"Ro*,Er*,PM*,Er*,V*,**,NIR,PM*,*,*,X ...",6.030,316.730266,38.742044
3,* tau Cyg,"PM*,dS*,PM*,PM*,dS*,V*,NIR,PM*,MIR,**,*,IR,UV,...",3.730,318.697886,38.045318
4,* 61 Cyg A,"Ro*,BY*,PM*,V*,PM*,Er*,NIR,**,*,*,IR,UV,X,X ...",5.210,316.724748,38.749417
5,* sig Cyg,"sg*,MIR,NIR,*,UV ...",4.240,319.353968,39.394680
...,...,...,...,...,...
8943,HD 77653A,"**,* ...",5.570,135.435644,-52.188689
8944,HD 141913A,"**,* ...",6.340,238.719320,-60.743652
8945,HD 56239A,* ...,6.190,108.008277,-63.190073


In [6]:
%reset_selective -f latitude
%reset_selective -f longitude
%reset_selective -f observation_time

latitude = 45.484264 # fixed latitude, modify or delete to be prompted to write it

if 'latitude' in locals():
    if latitude < -90 or latitude > 90:
        raise ValueError("Latitude must be between -90 and 90 degrees.")
        
if 'latitude' not in locals():
    while True:
        try:
            latitude = float(input("Enter your latitude: "))
            if -90 < latitude < 90:
                break
            else:
                print("Invalid input. Latitude must be between -90 and 90.")
        except ValueError:
            print("Invalid input. Please enter a numeric value for latitude.")

            
longitude = 15.96223 # fixed longitude, modify or delete to be prompted to write it

if 'longitude' in locals():
    if longitude < -180 or longitude > 180:
        raise ValueError("Longitude must be between -180 and 180 degrees (negative is western hemisphere, positive is eastern).")
 
if 'longitude' not in locals():
    while True:
        try:
            longitude = float(input("Enter your longitude: "))
            if -180 < longitude < 180:
                break
            else:
                print("Invalid input. Longitude must be between -180 and 180 (negative is western hemisphere, positive is eastern).")
        except ValueError:
            print("Invalid input. Please enter a numeric value for longitude.")


observation_time = Time("2024-08-08 21:00:00") # fixed observation time in UTC, modify or delete to be prompted to write it            
            
if 'observation_time' not in locals():            
    while True:
        date_time = input("Enter date and time in UTC (YYYY-MM-DD HH:MM format): ")
        date_time_pattern = re.compile(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}$')
        if date_time_pattern.match(date_time):
            year, month, day, hour, minute = map(int, date_time.split()[0].split('-') + date_time.split()[1].split(':'))
            if (1 <= month <= 12 and
                1 <= day <= 31 and
                0 <= hour <= 23 and
                0 <= minute <= 59):
                break
            else:
                print("Invalid input. Please enter a valid date and time.")
        else:
            print("Invalid input. Please enter a date and time in the YYYY-MM-DD HH:MM format.")
    date_object = datetime.strptime(date_time, '%Y-%m-%d %H:%M')
    observation_time = Time(date_object) 


observer_location = EarthLocation(lat=latitude*u.deg, lon=longitude*u.deg)
star_coords = SkyCoord(ra=df['RA'].values*u.deg, dec=df['Dec'].values*u.deg)

try:
    altaz = star_coords.transform_to(AltAz(obstime=observation_time, location=observer_location))
    df['az'] = altaz.az[:].degree
    df['alt'] = altaz.alt[:].degree
except (ValueError, RuntimeWarning) as e:
    print(f"An error occurred during coordinate transformation: {e}")
    df['az'] = np.nan
    df['alt'] = np.nan
    
    
altaz = star_coords.transform_to(AltAz(obstime=observation_time, location=observer_location))
df['az'] = altaz.az[:].degree
df['alt'] = altaz.alt[:].degree

Add coordinates of visible Solar System bodies:

In [11]:
print('Coordinates of Solar System bodies:')

sun_altaz = get_sun(observation_time).transform_to(AltAz(obstime=observation_time, location=observer_location))
observation_time = Time(observation_time, format='isot', scale='utc')

sun_altaz = get_sun(observation_time).transform_to(AltAz(obstime=observation_time, location=observer_location))
sun_altitude_deg = sun_altaz.alt.degree
sun_azimuth_deg = sun_altaz.az.degree

print("\nSun altitude: ", sun_altitude_deg, '° (', sun_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Sun azimuth: ", sun_azimuth_deg, '° (', sun_altaz.az.to_string(unit=u.degree),')', sep='')

moon_altaz = get_body('moon', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
moon_altitude_deg = moon_altaz.alt.degree
moon_azimuth_deg = moon_altaz.az.degree

print("\nMoon altitude: ", moon_altitude_deg, '° (', moon_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Moon azimuth: ", moon_azimuth_deg, '° (', moon_altaz.az.to_string(unit=u.degree),')', sep='')

mercury_altaz = get_body('mercury', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
mercury_altitude_deg = mercury_altaz.alt.degree
mercury_azimuth_deg = mercury_altaz.az.degree

print("\nMercury altitude: ", mercury_altitude_deg, '° (', mercury_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Mercury azimuth: ", mercury_azimuth_deg, '° (', mercury_altaz.az.to_string(unit=u.degree),')', sep='')

venus_altaz = get_body('venus', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
venus_altitude_deg = venus_altaz.alt.degree
venus_azimuth_deg = venus_altaz.az.degree

print("\nVenus altitude: ", venus_altitude_deg, '° (', venus_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Venus azimuth: ", venus_azimuth_deg, '° (', venus_altaz.az.to_string(unit=u.degree),')', sep='')

mars_altaz = get_body('mars', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
mars_altitude_deg = mars_altaz.alt.degree
mars_azimuth_deg = mars_altaz.az.degree

print("\nMars altitude: ", mars_altitude_deg, '° (', mars_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Mars azimuth: ", mars_azimuth_deg, '° (', mars_altaz.az.to_string(unit=u.degree),')', sep='')

jupiter_altaz = get_body('jupiter', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
jupiter_altitude_deg = jupiter_altaz.alt.degree
jupiter_azimuth_deg = jupiter_altaz.az.degree

print("\nJupiter altitude: ", jupiter_altitude_deg, '° (', jupiter_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Jupiter azimuth: ", jupiter_azimuth_deg, '° (', jupiter_altaz.az.to_string(unit=u.degree),')', sep='')

saturn_altaz = get_body('saturn', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
saturn_altitude_deg = saturn_altaz.alt.degree
saturn_azimuth_deg = saturn_altaz.az.degree

print("\nSaturn altitude: ", saturn_altitude_deg, '° (', saturn_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Saturn azimuth: ", saturn_azimuth_deg, '° (', saturn_altaz.az.to_string(unit=u.degree),')', sep='')

uranus_altaz = get_body('uranus', observation_time, observer_location).transform_to(AltAz(obstime=observation_time, location=observer_location))
uranus_altitude_deg = uranus_altaz.alt.degree
uranus_azimuth_deg = uranus_altaz.az.degree

print("\nUranus altitude: ", uranus_altitude_deg, '° (', uranus_altaz.alt.to_string(unit=u.degree),')', sep='')
print("Uranus azimuth: ", uranus_azimuth_deg, '° (', uranus_altaz.az.to_string(unit=u.degree),')', sep='')

Coordinates of Solar System bodies:

Sun altitude: -22.802544220571637° (-22d48m09.15919406s)
Sun azimuth: 328.09143804618645° (328d05m29.17696627s)

Moon altitude: -11.845866920617128° (-11d50m45.12091422s)
Moon azimuth: 278.30937533491937° (278d18m33.75120571s)

Mercury altitude: -24.631763016875254° (-24d37m54.34686075s)
Mercury azimuth: 309.32133049485043° (309d19m16.78978146s)

Venus altitude: -18.662164852097256° (-18d39m43.79346755s)
Venus azimuth: 309.45020267637915° (309d27m00.72963496s)

Mars altitude: -14.591645836135575° (-14d35m29.92501009s)
Mars azimuth: 35.530369631246316° (35d31m49.33067249s)

Jupiter altitude: -15.704265936066335° (-15d42m15.35736984s)
Jupiter azimuth: 32.78350849204557° (32d47m00.63057136s)

Saturn altitude: 14.698438649362547° (14d41m54.37913771s)
Saturn azimuth: 115.93694687894013° (115d56m13.00876418s)

Uranus altitude: -8.85433046891216° (-8d51m15.58968808s)
Uranus azimuth: 50.65321173464225° (50d39m11.56224471s)


Calculate the ecliptic line for the given time and location:

In [7]:
A = 23.5
T = 24
omega = 2 * np.pi / T

t = np.arange(0, 24, 0.01)
y = A * np.sin(omega * t)

y_dec = y*u.deg
t_ra = t*u.hour

coords = SkyCoord(ra=t_ra, dec=y_dec, frame='icrs')

altaz_frame = AltAz(obstime=observation_time, location=observer_location)
altaz_coords = coords.transform_to(altaz_frame)

altitudes = altaz_coords.alt.degree
azimuths = altaz_coords.az.degree

Calculate the boundary for circumpolar stars:

In [8]:
x_circ = np.arange(0, 24, 0.01)

if latitude > 0:
    y_circ=np.empty(2400); y_circ.fill(90 - latitude)
else:
    y_circ=np.empty(2400); y_circ.fill(-90 - latitude)
    
x_min = x_circ[0]
x_max = x_circ[-1]

decl_circ = y_circ*u.deg
righta_circ = x_circ*u.hour

coords_circ = SkyCoord(ra=righta_circ, dec=decl_circ, frame='icrs')

altaz_frame = AltAz(obstime=observation_time, location=observer_location)
altaz_coords_circ = coords_circ.transform_to(altaz_frame)

altitudes_circ = altaz_coords_circ.alt.degree
azimuths_circ = altaz_coords_circ.az.degree

Define the sizes of the stars:

In [9]:
def get_point_size(mag_v): # size of the stars
    if mag_v < 0:
        return 40
    if mag_v < 1:
        return 18
    elif 1 <= mag_v < 2:
        return 9
    elif 2 <= mag_v < 3:
        return 4
    elif 3 <= mag_v < 4:
        return 0.5
    elif 4 <= mag_v < 5:
        return 0.1
    else:
        return 0.02
    
df['point_size'] = df['Mag_V'].apply(get_point_size)

Calculate the illumination of the Moon at the given time:

In [10]:
time = Time(observation_time, format='iso', scale='utc')
moon_illumination = astroplan.moon_illumination(time)
moon_illumination_percentage = moon_illumination * 100

Plot the sky:

In [12]:
def plot_data_with_slider(Vmag_threshold):
    df_visible = df[df['alt'] >= 0]
    subset_df = df_visible[df_visible['Mag_V'] <= Vmag_threshold]
    print(f'Threshold magnitude: {Vmag_threshold}, visible stars: {len(subset_df)}')
    
    fig, axs = plt.subplots(1, 1, figsize=(15,10), subplot_kw={'projection': 'polar'})
   
    axs.set_rlim(bottom=90, top=0)
    axs.scatter(np.radians(subset_df['az']), subset_df['alt'], s=subset_df['point_size'], label='Stars')
    
    axs.set_theta_zero_location('N')
    axs.set_theta_direction(1)
    axs.set_rlabel_position(-90)
    plt.legend()
    
    observation_time_str = observation_time.strftime('%d/%m/%Y %H:%M')
    if latitude > 0:
        lat_str = 'N'
    else:
        lat_str = 'S'
    if longitude > 0:
        lon_str = 'E'
    else:
        lon_str = 'W'
    
    axs.set_xlabel('Azimuth')
    axs.set_ylabel('Altitude', rotation=90, labelpad=30)
    axs.yaxis.set_label_position('right') 

    declination_ticks = np.arange(90, -10, -15) 
    declination_tick_labels = [f"{declination}°" for declination in declination_ticks]
    axs.set_yticks(declination_ticks)
    axs.set_yticklabels(declination_tick_labels)
    
    axs.set_xticks(np.radians(np.arange(0, 360, 15)))  
        
    axs.set_title(f'Sky at {observation_time_str} UTC for {np.abs(latitude)}°{lat_str} and {np.abs(longitude)}°{lon_str}', fontsize=20)

    if sun_altaz.alt > 0:    
        axs.scatter(np.radians(sun_altaz.az), sun_altaz.alt, s=400, label='Sun', color='yellow')
    if moon_altaz.alt > 0:
        axs.scatter(np.radians(moon_altaz.az), moon_altaz.alt, s=100, label=f'Moon ({moon_illumination_percentage:.2f}%)', color='silver')
    if mercury_altaz.alt > 0:    
        axs.scatter(np.radians(mercury_altaz.az), mercury_altaz.alt, s=40, label='Mercury', color='lightsteelblue')
    if venus_altaz.alt > 0:    
        axs.scatter(np.radians(venus_altaz.az), venus_altaz.alt, s=40, label='Venus', color='lightgreen')  
    if mars_altaz.alt > 0:    
        axs.scatter(np.radians(mars_altaz.az), mars_altaz.alt, s=40, label='Mars', color='red')
    if jupiter_altaz.alt > 0:    
        axs.scatter(np.radians(jupiter_altaz.az), jupiter_altaz.alt, s=80, label='Jupiter', color='brown')
    if saturn_altaz.alt > 0:    
        axs.scatter(np.radians(saturn_altaz.az), saturn_altaz.alt, s=60, label='Saturn', color='sandybrown')
    if uranus_altaz.alt > 0:    
        axs.scatter(np.radians(uranus_altaz.az), uranus_altaz.alt, s=40, label='Uranus', color='cyan')

    iers.conf.auto_download = True

    plot_ecliptic = True
    if plot_ecliptic:
        axs.plot(np.radians(azimuths), altitudes, label='Ecliptic line', color='green',linewidth=1)
    plot_circumpolar = True
    if plot_circumpolar:
        axs.plot(np.radians(azimuths_circ), altitudes_circ, label='Circumpolar stars boundary', color='black',linewidth=1, linestyle='--')
    
    plt.legend(loc='best')
    plt.show()

Vmag_min = -1
Vmag_max = 6.5

Vmag_slider = FloatSlider(value=Vmag_max, min=Vmag_min, max=Vmag_max, step=0.5, description='Limiting magnitude', layout={'width': '400px'},style={'description_width': 'initial'})

interact(plot_data_with_slider, Vmag_threshold=Vmag_slider)

plt.show()

interactive(children=(FloatSlider(value=6.5, description='Limiting magnitude', layout=Layout(width='400px'), m…